In [55]:
## betting loop 
from pydantic import BaseModel

# NEED TO KEEP TRACK OF THOSE WHO FOLD

class Player(BaseModel):
    pid : int
    funds : int = 100
    active_in_current_hand : bool = True
    def make_bet(self, pot_state) -> (str, int):
        print(f"Player{self.pid}, given {pot_state}, your choices are:")
        print("  1)fold")
        print(f"  2)call for {pot_state['call_amount']}")
        print(f"  3)raise for at least {pot_state['call_amount']*2}")
        amount = input()
        ## if raise 
        pot_state['call_amount'] = int(amount)
    
PLAYERS  = [Player(pid = i) for i in range(0,3)]

class Pot(BaseModel):
    pot_state :int = {"size" : 0, "call_amount": 10}

    def betting_round(self):
        for player in PLAYERS:
            if player.active_in_current_hand:
                player.make_bet(pot_state)
                # if raise restart the loop... maybe a heap? stack?





    



In [56]:
Pot().betting_round()
    

Player0, given {'size': 0, 'call_amount': '3'}, your choices are:
  1)fold
  2)call for 3
  3)raise for at least 33


 2


Player1, given {'size': 0, 'call_amount': 2}, your choices are:
  1)fold
  2)call for 2
  3)raise for at least 4


 2


Player2, given {'size': 0, 'call_amount': 2}, your choices are:
  1)fold
  2)call for 2
  3)raise for at least 4


 2
